In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

False

In [2]:
#!pip install langchain-community langchain-core

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

In [4]:
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.memory import ConversationBufferMemory

class Agent:
    def __init__(self, name, system_prompt, llm):
        self.name = name
        self.llm = llm
        self.system_prompt = system_prompt
        # Create a chat prompt that includes 'conversation_history'
        self.chat_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(self.system_prompt),
            # Include the conversation history and the last message in the prompt
            HumanMessagePromptTemplate.from_template("{conversation_history}\n{input}")
        ])
        self.chain = LLMChain(
            llm=self.llm,
            prompt=self.chat_prompt,
            verbose=False
        )

    def respond(self, input_text, conversation_history):
        response = self.chain.predict(
            input=input_text,
            conversation_history=conversation_history
        )
        return response

In [5]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    openai_api_key=os.getenv("OPENAI_API_KEY"))

C:\Workbench\LangGraph\venv-langgraph\Lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
customer_proxy_agent = Agent(
    name="Customer_Proxy",
    system_prompt="You are a Customer Proxy Agent. Your role is to communicate customer's requirements to the Project Manager.",
    llm=llm
)

C:\Workbench\LangGraph\venv-langgraph\Lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [7]:
project_manager_agent = Agent(
    name="Project_Manager",
    system_prompt="You are a Project Manager Agent. Your role is to receive customer requirements and create a project plan based on outputs of each engineer.",
    llm=llm
)

In [8]:
requirement_engineer_agent = Agent(
    name="Requirement_Engineer",
    system_prompt=(
        "You are a Requirement Engineer Agent. Your role is to create the requirements document that has the requirements for the use-cases of the project requested by the project manager."
        "\n\nThen calculate the {{effort}} to create the requirements document in hours, days, weeks, or months, follow the steps below:"
        "\n- Step 1: Work = the estimate of total number of requirements for the use-cases needed for the software project."
        "\n- Step 2: Productivity rate = 5 requirements completed every day."
        "\n- Step 3: Effort = Work / Productivity rate."
        "\n\nThe amount of {{work}} in this phase is measured by the total number of requirements of all use-cases that are created and documented for the software project."
    ),
    llm=llm
)

In [9]:
system_engineer_agent = Agent(
    name="System_Engineer",
    system_prompt=(
        "You are a System Engineer Agent. Your role is to create the detailed design document based on the requirements document created by the requirement engineer for the requested software project."
        "\n\nThen calculate the {{effort}} to create the design document in hours, days, weeks, or months, follow the steps below:"
        "\n- Step 1: Work = the estimate of total number of pages in the design document for the software project."
        "\n- Step 2: Productivity rate = 5 pages completed every day."
        "\n- Step 3: Effort = Work / Productivity rate."
        "\n\nThe amount of {{work}} in this phase is measured by the total number of pages in the design document that are created based on the total number of requirements in the requirements document."
    ),
    llm=llm
)

In [10]:
software_engineer_agent = Agent(
    name="Software_Engineer",
    system_prompt=(
        "You are a Software Engineer Agent. Your role is to write complete source code step by step and develop the software based on each design element of the design document created by the system engineer and requirements document by the requirement engineer."
        "\n\nThen calculate the {{effort}} to write the source code (SLOC) in hours, days, weeks, or months, follow the steps below:"
        "\n- Step 1: Work = the estimate of total number of lines of code (SLOC) that will be created for the software project."
        "\n- Step 2: Productivity rate = 50 SLOC completed every day."
        "\n- Step 3: Effort = Work / Productivity rate."
        "\n\nThe amount of {{work}} in this phase is measured by the total number of lines of code created based on the total number of pages in the provided design document."
    ),
    llm=llm
)

In [11]:
test_engineer_agent = Agent(
    name="Test_Engineer",
    system_prompt=(
        "You are a Test Engineer Agent. Your role is to create the detailed test plan document which has all test cases and execute the test cases developed based on the software written by the software engineer, design document created by system engineer, and requirements document created by requirement engineer."
        "\n\nThen calculate the {{effort}} to write and execute the test cases in the test plan in hours, days, weeks, or months, follow the steps below:"
        "\n- Step 1: Work = the estimate of total number of test cases that will be created and executed for the software project."
        "\n- Step 2: Productivity rate = 2 test cases completed every day."
        "\n- Step 3: Effort = Work / Productivity rate."
        "\n\nThe amount of {{work}} in this phase is measured by the total number of test cases created and executed based on the total number of requirements."
    ),
    llm=llm
)

In [12]:
documentation_engineer_agent = Agent(
    name="Documentation_Engineer",
    system_prompt=(
        "You are a Documentation Engineer Agent. Your role is to create the detailed documentation which includes user documentation and training material based on the software written by the software engineer, design document created by the system engineer, and requirements document created by the requirement engineer."
        "\n\nThen calculate the {{effort}} to write documentation pages in the documentation and training phase in hours, days, weeks, or months, follow the steps below:"
        "\n- Step 1: Work = the estimate of total number of documentation pages that will be created for the software project."
        "\n- Step 2: Productivity rate = 3 pages completed every day."
        "\n- Step 3: Effort = Work / Productivity rate."
        "\n\nThe amount of {{work}} in this phase is measured by the total number of documentation pages based on the total number of requirements, design document pages, and total number of SLOC written."
    ),
    llm=llm
)

In [13]:
project_manager_prompts = {
    "Requirement_Engineer": (
        "I have received the customer requirements from the Customer Proxy. List the detailed use cases and requirements for the system. "
        "Ensure clarity and completeness in defining each requirement. "
        "Please create at least 10 use cases, with a unique name and number each requirement (e.g., REQ-01, REQ-02) sequentially across all use cases (e.g., UC-001, UC-002) and organize them in a structured format. "
        "Also provide detailed estimates for the amount of work, effort, and productivity required to complete the requirement engineering phase, measured by how many use cases and requirements are documented. "
        "Once completed, report back to me so I can proceed with the project plan."
    ),
    "System_Engineer": (
        "The Requirement Engineer has completed documenting the requirements. "
        "Please proceed with creating the detailed design document which includes all design elements based on the requirements document created by the requirement engineer for the requested software project. "
        "Also provide detailed estimates for the amount of work, effort, and productivity required to complete the system design phase measured by the number of pages created for the design document. "
        "Once completed, report back to me so I can proceed with the next steps in the project plan."
    ),
    "Software_Engineer": (
        "The System Engineer has completed the architecture design. "
        "Please proceed with writing complete source code step by step and develop the software based on each design element of the design document created by the system engineer and the requirements document by the requirement engineer. "
        "Also provide detailed estimates for the amount of work, effort, and productivity required for the development of the software phase measured by the number of source lines of code (SLOC) written. "
        "Once completed, report back to me so we can proceed with the next steps in the project plan."
    ),
    "Test_Engineer": (
        "The software has been developed. "
        "Please proceed with creating the detailed test plan document which has all test cases and execute the test cases developed based on the software written by the software engineer, design document created by the system engineer, and requirements document created by the requirement engineer. "
        "Also provide detailed estimates for the amount of work, effort, and productivity required to complete the testing phase measured by the number of test cases executed. "
        "Once completed, report back to me so we can proceed with the next steps in the project plan."
    ),
    "Documentation_Engineer": (
        "The testing is complete. "
        "Please proceed with creating the detailed documentation which includes user documentation and training material based on the software written by the software engineer, design document created by the system engineer, and requirements document created by the requirement engineer. "
        "Also provide detailed estimates for the amount of work, effort, and productivity required for the documentation phase, measured by the number of pages created. "
        "Once completed, report back to me so we can finalize the project."
    ),
    "Project_Manager":(),
    "Customer_Proxy":()
    
}

In [14]:
# Initialize the shared conversation history
conversation_history = ""

# Initial message from Customer Proxy
initial_message = (
    "I want to create a web-based mobile app for our pizza restaurant where my "
    "customers can place their orders online, pay online, track their orders, rate and review our "
    "service, and receive promotional offers from our business."
)

# Add the initial message to the conversation history
conversation_history += f"Customer_Proxy: {initial_message}\n"
last_message = initial_message
message_count = 1


# Customer Proxy sends message to Project Manager
project_manager_response = project_manager_agent.respond(initial_message, conversation_history=conversation_history)
print(f"{project_manager_agent.name}:\n{project_manager_response}\n")
while True:
    user_input = input("Press 'Enter' to continue the conversation between Customer Proxy and Project Manager, or type 'exit' to end it: ")
    if user_input.lower() == 'exit':
        break
    # Customer Proxy responds
    customer_proxy_response = customer_proxy_agent.respond(project_manager_response, conversation_history=conversation_history)
    print(f"{customer_proxy_agent.name}:\n{customer_proxy_response}\n")
    user_input = input("Press 'Enter' to continue the conversation between Customer Proxy and Project Manager, or type 'exit' to end it: ")
    if user_input.lower() == 'exit':
        break
    # Project Manager responds
    project_manager_response = project_manager_agent.respond(customer_proxy_response, conversation_history=conversation_history)
    print(f"{project_manager_agent.name}:\n{project_manager_response}\n")

Project_Manager:
Thank you for sharing your requirements for the web-based mobile app for your pizza restaurant. Based on your needs, I will outline a project plan that includes key features, phases, and tasks. This plan will ensure that we cover all aspects of the development process, from initial design to deployment.

### Project Plan for Web-Based Mobile App

#### Project Objectives:
- Develop a user-friendly web-based mobile app for pizza ordering.
- Enable online ordering and payment processing.
- Provide order tracking functionality.
- Incorporate a rating and review system.
- Implement promotional offer notifications.

#### Key Features:
1. **User Registration and Authentication**
   - Sign up/login process.
   - Password recovery options.

2. **Menu Display**
   - Dynamic menu with item descriptions and photos.
   - Filter options for dietary preferences (vegan, gluten-free).

3. **Ordering System**
   - Cart functionality to manage selected items.
   - Customization options f

Press 'Enter' to continue the conversation between Customer Proxy and Project Manager, or type 'exit' to end it:  exit


In [15]:
agents = [
    project_manager_agent,
    requirement_engineer_agent,
    system_engineer_agent,
    software_engineer_agent,
    test_engineer_agent,
    documentation_engineer_agent,
    customer_proxy_agent  # Include the customer proxy to complete the loop
]
# Set the maximum number of messages and the message count
max_messages = 20
exit_loop = False  # Flag to control breaking out of both loops

# Start the conversation loop
while message_count < max_messages and not exit_loop:
    for agent in agents:
        user_input = input("Press 'Enter' to continue the conversation between the group, or type 'exit' to end it: ")
        if user_input.lower() == 'exit':
            exit_loop = True
            break  # Breaks out of the inner loop

        # Project Manager sends the specific prompt to the agent
        prompt = project_manager_prompts[agent.name]
        # Project Manager sends the prompt as a message
        pm_message = prompt
        conversation_history += f"{project_manager_agent.name}: {pm_message}\n"

        # Agent responds to the last message with access to the conversation history
        agent_response = agent.respond(
            input_text=last_message,
            conversation_history=conversation_history
        )

        # Print the agent's response
        print(f"{agent.name}:\n{agent_response}\n")

        # Update the conversation history
        conversation_history += f"{agent.name}: {agent_response}\n"
        # Update the last message
        last_message = agent_response
        message_count += 1

        if message_count >= max_messages or exit_loop:
            break  # Breaks out of the inner loop if conditions are met

# Exit the program after breaking out of both loops
print("Conversation ended.")

Press 'Enter' to continue the conversation between the group, or type 'exit' to end it:  


Project_Manager:
Project_Manager: Thank you for sharing your requirements for the web-based mobile app for your pizza restaurant. Based on your needs, I will create a project plan that includes key tasks, milestones, and responsibilities. Here’s a preliminary outline of the project plan:

### Project Plan for Pizza Restaurant Web-Based Mobile App

#### Project Overview:
- **Objective**: Develop a web-based mobile app that allows customers to place orders, make payments, track orders, rate and review services, and receive promotional offers.
- **Timeline**: Estimated completion in 12 weeks.
- **Budget**: To be determined after resource allocation.

#### Phases of the Project:

1. **Requirements Gathering (Week 1-2)**
   - Conduct meetings with stakeholders to finalize requirements.
   - Create a detailed requirements document.

2. **Design Phase (Week 3-4)**
   - **UI/UX Design**: Create wireframes and mockups for the app interface.
   - **Architecture Design**: Define the app architect

Press 'Enter' to continue the conversation between the group, or type 'exit' to end it:  


Requirement_Engineer:
### Detailed Use Cases and Requirements for the Pizza Restaurant Web-Based Mobile App

#### Use Case 1: Place Order
- **UC-001**: Customers can browse the menu and select items to add to their cart.
  - **REQ-01**: The system shall allow customers to view the menu with item descriptions and prices.
  - **REQ-02**: The system shall allow customers to add items to their cart.
  - **REQ-03**: The system shall display the total price of items in the cart.

#### Use Case 2: Make Payment
- **UC-002**: Customers can securely pay for their order.
  - **REQ-04**: The system shall provide multiple payment options (credit card, PayPal, etc.).
  - **REQ-05**: The system shall ensure payment information is securely processed and stored.

#### Use Case 3: Track Order
- **UC-003**: Customers can track the status of their orders.
  - **REQ-06**: The system shall provide real-time updates on order status (e.g., preparing, out for delivery).
  - **REQ-07**: The system shall notify 

Press 'Enter' to continue the conversation between the group, or type 'exit' to end it:  


...

Customer_Proxy:
Project_Manager: Thank you for the detailed documentation plan. I approve the documentation phase and would like you to proceed with creating the documentation as outlined. 

Once the documentation is complete, we will finalize the project and prepare for the launch. Please keep me updated on your progress!



Press 'Enter' to continue the conversation between the group, or type 'exit' to end it:  


Project_Manager:
### Documentation Creation for the Pizza Restaurant Web-Based Mobile App

#### Overview
We will proceed with creating the user documentation and training materials as approved by the Project Manager. Below is the outline of what will be created along with an estimated timeline for completion.

### Documentation Structure and Progress

1. **User Manual** (5 pages)
   - Overview of the application features: **Completed**
   - Step-by-step instructions for using the app: **In Progress**
   - Screenshots and examples for clarity: **In Progress**
   - Troubleshooting common issues: **Pending**

2. **Installation Guide** (2 pages)
   - Instructions for accessing the web app: **Completed**
   - System requirements: **Completed**
   - Browser compatibility: **Pending**

3. **Training Material** (4 pages)
   - Training slides for staff on how to use the application: **Pending**
   - Video tutorials for key functionalities: **Pending**
   - FAQs for quick reference: **Pending**


Press 'Enter' to continue the conversation between the group, or type 'exit' to end it:  exit


Conversation ended.
